# Prepare the notebook for use

## Install packages & import libraries

In [1]:
# Download required packages to Colab
!pip install -q langchain
!pip install -q openai
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.4 MB/s eta

In [ ]:
# Import required libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain import ConversationChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import os
from getpass import getpass

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

import pandas as pd
import csv

## OpenAI API key

In [3]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


## Connect Google Drive

## Read data

1. [var_comb] varibale_combination.csv
2. [prompt_result] answer_evaluation_prompt_result.csv
3. [qa_comb] interview_question_answer_combination.csv
4. [how_to_answer] how_to_answer_behavioral_question.txt
5. [company_looking] what_are_things_companies_looking_for_during_behavioral_interview.txt

Since the results we got from LLMs will be different everytime, we want to use .csv files to track those results which could help use go back to see what we can do or what we did to improve the outputs.

In [46]:
var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")
prompt_result = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/answer_evaluation_prompt_result.csv")
qa_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/interview_question_answer_combination.csv")

In [47]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description
0,1,recruiter,S&P Global,\nEDUCATION\nVanderbilt University – Data Scie...,Data Engineer - AI/ML (Virtual),\nJob Description\n\nThis is a software develo...,\nOverview\nS&P Global (NYSE: SPGI) provides e...


In [48]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...


In [49]:
qa_comb

,comb_num,var_comb_num,interview_question,question_type,question_answer
0,1,1,Describe a challenging project you worked on. ...,behavioral,"\nIn my last role as a project manager, I led ..."
1,2,1,Describe a challenging project you worked on. ...,behavioral,\nI was part of a marketing campaign that face...
2,3,1,Describe a challenging project you worked on. ...,behavioral,"\nAs a junior analyst, I worked on a data migr..."
3,4,1,Describe a challenging project you worked on. ...,behavioral,"\nI worked on a challenging project last year,..."
4,5,1,Describe a challenging project you worked on. ...,behavioral,"\nIn my previous job, there was a project that..."
5,6,1,Describe a challenging project you worked on. ...,behavioral,\nI was involved in a project that didn't go w...
6,7,1,Describe a challenging project you worked on. ...,behavioral,\nI led a cross-functional team on a critical ...
7,8,1,Describe a challenging project you worked on. ...,behavioral,"\nIn my role as an IT specialist, I was tasked..."
8,9,1,How do you handle disagreements with colleagues?,behavioral,\nI believe in open communication and empathy....
9,10,1,How do you handle disagreements with colleagues?,behavioral,\nDisagreements can be challenging. I usually ...


## Define Parameters

In [50]:
var_comb_num = 1 # this means we use the 1st combination of information in var_comb.csv dataset; this helps us tack information

In [51]:
file1 = open("/content/drive/MyDrive/Capstone Project - Mock Interview App/data/how_to_answer_behavioral_question.txt", "r")
file2 = open("/content/drive/MyDrive/Capstone Project - Mock Interview App/data/what_are_things_companies_looking_for_during_behavioral_interview.txt", "r")

In [52]:
how_to_answer = file1.read()
company_looking = file2.read()

In [53]:
# print(how_to_answer)
# print("===============")
# print(company_looking)

# Model selection

In [54]:
chat = ChatOpenAI(model_name = "gpt-4")

In [55]:
# record model as ex_model
ex_model = "gpt_4"

# Answer Evaluation Prompt

## Prompt 2.3 (var with detail)

In [56]:
# set up input variables for prompt
input_var_4 = ["interview_questions",
               "role_title",
               "company_name",
               "question_answers",
               "role_description", "company_description"]

In [58]:
template_4 = """
As an experienced career coach, you help students improve their skills for
answering behavioral interview questions.  Please help this student improve the
answer to the behavioral interview question {interview_questions} during the
interview of {role_title} at {company_name}.

The student’s answer: {question_answers}

First, give the student's answer a score;
Second, give feedback for the student's answer, point by point with rationale based on 10 aspects of how interviewers should answer a behavioral interverview question;
Third, give your improved answer;
Fourth, give a score for your improved answer and the rationale of why it is better.

Here are job description and company backgroound that you might want to consider:
Job Description: {role_description}
Company Background: {company_description}

Here are aspects that how interviewers should answer a behavioral interview question:
When answering behavioral interview questions, it's essential to present your qualifications effectively by incorporating specific aspects that highlight your skills, experiences, and personal attributes. These aspects are crucial in demonstrating your suitability for the role and how well you align with the organization's values and culture. Here are key aspects to include:
1.	Specific Examples: Use concrete examples from your past experiences to illustrate how you've handled situations relevant to the question. This provides tangible evidence of your skills and abilities.
2.	STAR Method: Structure your response using the STAR method – Situation, Task, Action, and Result. This helps in organizing your answer clearly and concisely:
•	Situation: Describe the context or background of the example you're using.
•	Task: Explain the task or challenge you were facing.
•	Action: Detail the specific actions you took to address the task or challenge.
•	Result: Share the outcomes of your actions, highlighting your contributions and any lessons learned.
3.	Relevance to the Role: Tailor your examples to demonstrate skills and attributes that are directly relevant to the job you're applying for. Show how your past experiences have prepared you for this specific role.
4.	Problem-Solving Skills: Highlight your ability to identify problems, think critically, and implement effective solutions. Employers value candidates who can tackle challenges proactively.
5.	Teamwork and Collaboration: Discuss how you work with others, emphasizing teamwork, communication, and interpersonal skills, especially if the role involves working closely with a team.
6.	Adaptability and Flexibility: Show your ability to adapt to new situations, learn from experiences, and remain flexible in the face of change or uncertainty.
7.	Leadership and Initiative: For roles that require leadership, illustrate your ability to lead, inspire, and motivate others. Even for non-leadership roles, showing initiative and the ability to take charge in appropriate situations can be beneficial.
8.	Results and Impact: Focus on the results and impact of your actions. Quantify your achievements where possible, as this adds credibility and a sense of scale to your accomplishments.
9.	Personal Development: Reflect on what you learned from the experience and how it contributed to your personal and professional growth.
10.	Cultural Fit: Convey how your values, work ethic, and personality align with the company's culture. Show that you not only have the skills for the job but also would thrive in the organization's environment.
Including these aspects in your behavioral interview responses can significantly strengthen your answers, showcasing your qualifications, experiences, and fit for the role.

Here are aspects that companies hope to get from your answer of a behavioral question:
Companies conduct behavioral interviews to assess various aspects of a candidate's past behavior, skills, and performance to predict their future behavior and suitability for the role in their organization. Here are some key elements that companies typically look for during behavioral interviews:
1.	Past Performance as an Indicator of Future Behavior: Companies believe that past behavior is a good predictor of future behavior. By understanding how you handled situations in the past, they can gauge how you might perform in similar situations in their organization.
2.	Problem-Solving and Decision-Making Skills: Employers look for candidates who can demonstrate effective problem-solving and decision-making skills. They are interested in how you approach challenges, analyze problems, and arrive at solutions.
3.	Teamwork and Collaboration: Since most roles require some level of teamwork, companies want to see how well you work with others. Your ability to collaborate, communicate effectively, and contribute to a team is crucial.
4.	Adaptability and Flexibility: With the fast pace of change in the workplace, companies seek candidates who can adapt to new situations and challenges quickly and effectively.
5.	Leadership Qualities: For roles that involve leading others, employers look for leadership potential. This includes your ability to motivate, guide, and influence others, even if the role is not a traditional leadership position.
6.	Work Ethic and Professionalism: Your approach to work, commitment to quality, and professional conduct are important to employers. They want to ensure you have a strong work ethic and can represent the company positively.
7.	Cultural Fit: Companies are keen on finding candidates who will fit well with their organizational culture. This includes sharing similar values, work styles, and the ability to thrive in the company’s work environment.
8.	Resilience and Stress Management: Employers may look for signs of resilience and your ability to handle stress or failure. They want employees who can remain productive and positive, even in challenging situations.
9.	Communication Skills: Effective communication is key in any role. Companies assess your ability to articulate ideas clearly, listen to others, and communicate in a way that is effective and appropriate for the workplace.
10.	Initiative and Self-motivation: Demonstrating that you are a self-starter who takes initiative is highly valued. Companies prefer candidates who show they can work independently and are motivated to achieve.
In summary, behavioral interviews are designed to evaluate a candidate's suitability not only in terms of skills and experience but also in terms of their personality, work habits, and potential fit within the company's culture and team dynamics.
"""

In [59]:
prompt_23 = PromptTemplate(
    input_variables = input_var_4,
    template = template_4
)

### Code

In [60]:
job_title = var_comb.role_title[0]
comp_name = var_comb.company_name[0]

job_des = var_comb.role_description[0]
comp_des = var_comb.company_description[0]

#### Q1 A1 [7/10]

In [61]:
# set up prameters
qa_comb_num = 1
inter_q = qa_comb.interview_question[0]
q_a = qa_comb.question_answer[0]

In [62]:
#from pydantic.typing import resolve_annotations

new_prompt = prompt_23.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a,
                              role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [63]:
print(result.content)

Student's Answer Score: 7/10

Feedback for the Student's Answer:

1. Specific Examples: You provided a specific example of a challenging project you worked on, which is good. However, as this is for a Data Engineer - AI/ML role, it would have been better to choose a project more relevant to the job. (Score: 0.5/1)

2. STAR Method: Your response was structured well in the STAR method. You described the situation, your tasks, the actions you took, and the result. (Score: 1/1)

3. Relevance to the Role: Your answer was not very relevant to the specific role of a Data Engineer - AI/ML. A more related example involving AI/ML would have been more appropriate. (Score: 0.5/1)

4. Problem-Solving Skills: You demonstrated problem-solving skills by organizing extra brainstorming sessions and delegating tasks based on individual strengths, which is commendable. (Score: 1/1)

5. Teamwork and Collaboration: Your answer showed your ability to work with a team, delegate tasks, and maintain open commun

##### store output

In [64]:
# ex_model, qa_comb_num, var_comb_num, template_4, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_4],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [65]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...


#### Q1 A5 [4/10]

In [66]:
# set up prameters
question_num = 1
answer_num = 5
ind = (question_num-1) * 8 + answer_num - 1


qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

In [67]:
inter_q

'Describe a challenging project you worked on. What was your role and how did you handle it?'

In [68]:
q_a

"\nIn my previous job, there was a project that was really tough. I found it \noverwhelming, so I mostly kept to the tasks I was comfortable with and didn't \nventure much beyond that.\n"

In [69]:
new_prompt = prompt_23.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a,
                              role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [70]:
print(result.content)

Student's Answer Score: 4/10

Feedback/Rationale:

1. Specific Examples: The student did provide an example, but it lacked detail and context. It wasn't clear what the project was about or why it was challenging.
2. STAR Method: This answer doesn't follow the STAR method. The situation is vague, the task isn't defined, the actions are limited, and there's no mention of the outcome or result.
3. Relevance to the Role: It's unclear how this experience relates to the role of a Data Engineer. The student didn't mention any technical skills or tasks associated with data engineering.
4. Problem-Solving Skills: The student didn't demonstrate any problem-solving skills. They actually admitted to avoiding the challenging aspects of the project.
5. Teamwork and Collaboration: There's no mention of teamwork or collaboration, which are important components of the role.
6. Adaptability and Flexibility: The student showed a lack of adaptability by sticking to tasks they were comfortable with instead

##### store output

In [71]:
# ex_model, qa_comb_num, var_comb_num, template_4, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_4],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [72]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...


#### Q3 A7 [8/10]

In [73]:
# set up prameters
question_num = 3
answer_num = 7
ind = (question_num-1) * 8 + answer_num - 1

qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

In [74]:
inter_q

'How do you stay motivated and manage your time effectively?'

In [75]:
q_a

'\nMy motivation comes from setting challenging yet achievable goals. I use a \ncombination of the Eisenhower Matrix for prioritizing tasks and the Pomodoro \nTechnique for focused work sessions. This approach helped me successfully manage \nmultiple projects simultaneously, leading to a 20% increase in efficiency in my \ndepartment.\n'

In [76]:
new_prompt = prompt_23.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a,
                              role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [77]:
print(result.content)

Student's Answer Score: 8/10

Feedback for the Student's Answer:

1. Specific Examples: You have provided a clear example of your time management strategy, which includes the use of the Eisenhower Matrix and Pomodoro Technique. This gives the interviewer an idea of how you manage your tasks. Score: 1/1

2. STAR Method: You have structured your answer well, but there could be more context on the situation (the specific projects you were managing) and more details on the action (how you used the time management techniques). Score: 0.75/1

3. Relevance to the Role: You successfully connected your efficient time management with your ability to manage multiple projects, a skill necessary for the role. However, you did not mention how your motivation aligns with the role's demands. Score: 0.75/1

4. Problem-Solving Skills: You addressed this indirectly by mentioning how your time management skills increased efficiency. However, more explicit mention of problem-solving would strengthen your a

##### store output

In [78]:
# ex_model, qa_comb_num, var_comb_num, template_4, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_4],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [79]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...


# Prompt 2.3 works better
More details:
Compare & Contrast Answer Evaluation Prompts & Results
(https://docs.google.com/document/d/1WBKhjGyOa_67jFeTWWVvkvCUikkREorMbxOtcF1rKQQ/edit?usp=sharing).

<font color="Green">Done:  </font>  
-[X] Try to combine result form prompt 2 & prompt 3, so we could have both detailed feedback and answer better aligned with the job and company the student applied.

# Save data
answer_evaluation_prompt_result.csv

In [80]:
#prompt_result
prompt_result.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/answer_evaluation_prompt_result.csv", index=False)

In [81]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...
